# Multiplayer Metaflow

## Sharing

Data scientist's on the same Metaflow deployment have read access to each other's Metaflow results. 
This is useful in a project because you can make incremental progress on each other results, without reinventing the wheel.
You can also compare approaches, results, and split work in a seamless way.

Two concepts and sharing features that you should understand to benefit most from Metaflow in a team setting, are namespaces and projects.

## Namespaces

[Namespaces](https://docs.metaflow.org/scaling/tagging#namespaces) help you to keep results organized. 
A flow runs in a namespace, and the Metaflow Client API retrieves results for you based on the active namespace.
Namespaces are not about security or access control, but for organizing results by the users who produce and analyze them.

In [1]:
from metaflow import namespace, Metaflow

In [2]:
namespace('user:sandbox')
Metaflow().flows

[Flow('RF_Flow_cloud'),
 Flow('Branch_Flow_Cloud'),
 Flow('Branch_Cloud_Step'),
 Flow('Branch_Cloud_Flow'),
 Flow('DivideByZeroFlow'),
 Flow('RetryFlow'),
 Flow('TimeoutFlow'),
 Flow('CatchDivideByZeroFlow'),
 Flow('TaxiFarePrediction')]

In [3]:
# Expect: the same result in this global (None) namespace as the user:sandbox namespace, 
    # because your sandbox user is the only user on this Metaflow deployment.
namespace(None)
Metaflow().flows

[Flow('RF_Flow_cloud'),
 Flow('Branch_Flow_Cloud'),
 Flow('Branch_Cloud_Step'),
 Flow('Branch_Cloud_Flow'),
 Flow('DivideByZeroFlow'),
 Flow('RetryFlow'),
 Flow('TimeoutFlow'),
 Flow('CatchDivideByZeroFlow'),
 Flow('TaxiFarePrediction')]

## The production namespace
There is a special namespace designated production. Production namespaces are important for a variety of reasons. For example, it helps you add authorization keys to the process of deploying to a valuable Flow's production namespace. 

To get started in production, find your `TaxiFarePrediction` Flow from the week 3 project and run this command with the file path changed to wherever you place the flow in your week 4 workspace.

In [4]:
# Watch the file path if you run in the terminal! It may be easiest to do `cd ./notebooks`, if you want to run the commands as is from the terminal.
# If you run this cell from the notebook, you should be able to use the path as is.

# This will take a minute or two the first time it has to build the conda environment.
# Find the namespace this flow is deployed in. Is it production? 🧐

! python ../flows/cloud/event_triggered_linear_regression_solo.py --environment=conda argo-workflows create

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
2023-11-12 08:11:44.794 Creating local datastore in current directory (/home/workspace/workspaces/full-stack-ml-metaflow-corise-week-4/notebooks/.metaflow)
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taxifareprediction to Argo Workflows...

The namespace of this production flow is
    production:taxifareprediction-0-ouyj
To analyze results of this production flow add this line in your notebooks:
    namespace("production:taxifareprediction-0-ouyj")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize taxifareprediction-0-ouyj
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping virtual environment(s) ...
    Micromamba ran into an error while set

In the output, you will see a line like `namespace("user:sandbox")` or `namespace("production:taxifareprediction-0-ouyj")`. 
Copy that line, paste it in the first line of the next cell and run it. Then 

In [5]:
namespace("production:taxifareprediction-0-ouyj")

'production:taxifareprediction-0-ouyj'

In [6]:
# run this cell to print your Argo URL
import json
res = json.load(open("/home/workspace/.metaflowconfig/config.json"))
print("Go to this URL:", res['SANDBOX_VSCODE_URL'].replace('vs', 'argo'), "and click on the second top left tab 'Workflow Templates'. Then come back here and trigger the flow.")

Go to this URL: https://argo-pw-323314244.outerbounds.dev and click on the second top left tab 'Workflow Templates'. Then come back here and trigger the flow.


In [7]:
! python ../flows/cloud/event_triggered_linear_regression_solo.py --environment=conda argo-workflows trigger

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow taxifareprediction triggered on Argo Workflows (run-id argo-taxifareprediction-bkgkx).
See the run in the UI at https://ui-pw-323314244.outerbounds.dev/TaxiFarePrediction/argo-taxifareprediction-bkgkx


In [8]:
# This cell will return an empty list for a minute or two after Argo UI shows the flow running.
Metaflow().flows
# Do you see a different set of flows than what you saw above with the global or user namespace. 

[Flow('TaxiFarePrediction')]

## Projects
The [`@project decorator`](https://docs.metaflow.org/production/coordinating-larger-metaflow-projects) is for production use cases. 

It makes available three classes of namespaces that will affect the behavior of a production deployment:
1. `user` is the default. It will deploy to a user-specific, private namespace. Use it for testing production deployments.
2. `test` denotes custom branches that can be shared amongst multiple users. Use it for deploying experimental versions that can run in parallel with production. Deploy custom branches with `--branch foo`.
3. `prod` denotes the global production namespace. Use it for deploying the official production version of the project. Deploy to production with `--production`. For multiple production variants, deploy custom branches with `--production --branch foo`.

You don't need to remember these, but they are useful to revisit once you have thought more about the requirements of your ML project.
For example, later in this week you will consider how to deploy multiple production variants to score a champion/challenger model on production traffic.

### Motivation
Consider the situation after you deploy `TaxiFarePrediction` to Argo that is running in production. 
The next time you deploy `TaxiFarePrediction` by running the same command you just did a few cells ago,
```sh
python ../flows/cloud/event_triggered_linear_regression.py --environment=conda argo-workflows create
```
it will overwrite the production `TaxiFarePrediction` flow. Clearly, we want more optionality to run experiments.

What do you do when the workflow starts performing well, and multiple people want to test their own production deployments without interfering with yours; or if, as a single developer, you want to experiment with multiple independent deployments of your workflow? How do you create a new workflow without overwriting the existing one? 

### Going to --production

Metaflow's `@project` decorator makes it easy to specifiy the production namespace. You can use this to deploy workflows in different namespaces, and specify a dedicated production branch that is not for anything experimental.

In [9]:
! python ../flows/cloud/event_triggered_linear_regression.py --environment=conda --production argo-workflows create

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: fullstack, Branch: prod
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying fullstack.prod.taxifareprediction to Argo Workflows...
It seems this is the first time you are deploying fullstack.prod.taxifareprediction to Argo Workflows.

A new production token generated.

The namespace of this production flow is
    production:mfprj-a7y3zg2tm4k4wauw-0-simf
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-a7y3zg2tm4k4wauw-0-simf")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-a7y3zg2tm4k4wauw-0-simf
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping virtual environment(s) ...


In [11]:
namespace("production:mfprj-a7y3zg2tm4k4wauw-0-simf")

'production:mfprj-a7y3zg2tm4k4wauw-0-simf'

In [13]:
! python ../flows/cloud/event_triggered_linear_regression.py --environment=conda --production argo-workflows trigger

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: fullstack, Branch: prod
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
    Argo Workflows error:
    The workflow fullstack.prod.taxifareprediction doesn't exist on Argo Workflows in namespace jobs-pw-323314244. Please deploy your flow first.

